
PODE SE ESQUECER:

- linkToExtractedText: já temos o texto extraido, e para alem disso atraves do *link principal* consegue-se chegar a este

In [83]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

# filter some data (repeated and alias not in extractedText)

because we used `&dedupValue=25&dedupField=url` and different aliases, we might have repeated data, so it's important to check for it

due to 250 req/ min i had to creade this retry function: """Each API has the following usage limits (thresholds), please check if you are exceeding these limits if you start receiving the HTTP response status Error 429 too many requests:

Arquivo.pt API (Full-text & URL search): 250 requests per minute"""

# EM PRINCIPIO ESTE JA DEVE FUNCIONAR !

## exprimentar e dps reescrever mais bonito e tal e já temos ent data03 ig

In [104]:
def fetch_with_retry(linkToExtractedText):
    # Infinite loop to handle retry logic in case of 429 Too Many Requests
    while True:
        response = requests.get(linkToExtractedText)
        status_code = response.status_code
        
        if status_code == 200:
            # If the request is successful (200 OK), return the extracted text
            soup = BeautifulSoup(response.content, "html.parser")
            return soup.get_text()
        elif status_code == 429:
            # Handle 429 Too Many Requests by reading the Retry-After header
            print(f"Too many requests. Retrying after {60} seconds...")
            time.sleep(60)  # Pause execution for the retry period
        elif status_code == 404:
            return 0
        else:
            # For any other status codes (e.g., 500, ...), print the status and break the loop
            print(f"Request failed with status code {status_code}. Link was {linkToExtractedText}")
            break

# Function to process each column
def process_cell(column, aliases):

    stats = {"404": 0, "duplicate": 0}

    filtered_column = []

    for row in aliases.index:
        filtered_cell = []
        seen_text = set()
        print(f"{row}", end = "; ")
        for i in column.loc[row]:
            
            # Extract text from 'linkToExtractedText'
            text = fetch_with_retry(i['linkToExtractedText'])
                

            # Skip if the text has already been processed
            if text in seen_text:
                stats["duplicate"] += 1
                continue

            elif not text: #ERROR 404
                stats["404"] += 1
                continue
            
            # Check if any alias is found in the text
            elif any(alias.lower() in text.lower() for alias in aliases.loc[row]):
                i["ExtractedText"] = text  # Add extracted text to the record
                
                # Remove unwanted fields
                i.pop('title', None)
                i.pop('snippet', None)
                i.pop('linkToExtractedText', None)
                
                # Append the processed record
                filtered_cell.append(i)
                
                # Mark this text as processed
                seen_text.add(text)

        filtered_column.append(filtered_cell)
                
    return filtered_column, stats


df = pd.read_parquet("data02.parquet")


start_time = datetime.now()
print(f"Starting: {start_time}")
for column in df.columns:
    if column == "aliases":
        pass
    else:
        print(f"\nProcessing {column}", end = ": ")
        df[column], stats = process_cell(df[column], df["aliases"])
end_time = datetime.now()
print(f"Ended: {end_time}.")
print(stats)

df.to_parquet("data03.parquet")

Starting: 2024-10-06 12:10:56.834492

Processing api.2000: Banco Comercial Português; Galp Energia; Too many requests. Retrying after 60 seconds...
EDP; Sonae; Too many requests. Retrying after 60 seconds...
Mota-Engil; 
Processing api.2003: Banco Comercial Português; Too many requests. Retrying after 60 seconds...
Galp Energia; Too many requests. Retrying after 60 seconds...
EDP; Too many requests. Retrying after 60 seconds...
Too many requests. Retrying after 60 seconds...
Sonae; Too many requests. Retrying after 60 seconds...
Too many requests. Retrying after 60 seconds...
Mota-Engil; 
Processing api.2006: Banco Comercial Português; Too many requests. Retrying after 60 seconds...
Galp Energia; Too many requests. Retrying after 60 seconds...
EDP; Sonae; Too many requests. Retrying after 60 seconds...
Too many requests. Retrying after 60 seconds...
Mota-Engil; 
Processing api.2009: Banco Comercial Português; Too many requests. Retrying after 60 seconds...
Too many requests. Retrying a

ReadTimeout: HTTPSConnectionPool(host='arquivo.pt', port=443): Read timed out. (read timeout=None)

# easier to do column by column !! and save each column every time

df03 = df02 copy

fazer funcao para aplicar so a x colunas, e seleciono uma a uma, mas tem a opcao de ser duas ao mesmo tempo ou todas de uma vez etc

In [61]:
#df.equals(df_test)

In [92]:
df.map(lambda x: len(x)) #90, 44

,aliases,api.2012
companies,,
Mota-Engil,2,538
